In [1]:
import keras
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

2021-12-20 14:48:18.429550: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-20 14:48:18.429597: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.5.0
2.5.2


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

NGRAMS = 2
EPOCHS = 15
YEAR = '2000'
#YEAR = '2010'

df = pd.read_csv('../data/census/census_%s.csv' % YEAR)
df.dropna(subset=['name'], inplace=True)
df.replace('(S)', 0, inplace=True)
df

,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
0,SMITH,1,2376206,880.85,880.85,73.35,22.22,0.40,0.85,1.63,1.56
1,JOHNSON,2,1857160,688.44,1569.30,61.55,33.80,0.42,0.91,1.82,1.50
2,WILLIAMS,3,1534042,568.66,2137.96,48.52,46.72,0.37,0.78,2.01,1.60
3,BROWN,4,1380145,511.62,2649.58,60.71,34.54,0.41,0.83,1.86,1.64
4,JONES,5,1362755,505.17,3154.75,57.69,37.73,0.35,0.94,1.85,1.44
...,...,...,...,...,...,...,...,...,...,...,...
151666,ZILK,150436,100,0.04,89753.41,90.00,9.00,0.00,0.00,0,0
151667,ZINNANTI,150436,100,0.04,89753.45,98.00,0.00,0.00,0.00,0,0
151668,ZITTERICH,150436,100,0.04,89753.48,98.00,0,0.00,0.00,0.00,0
151669,ZULU,150436,100,0.04,89753.52,6.00,90.00,0.00,0.00,0,0


## Resampling with weight

In [3]:
sdf = df.sample(1000000, weights=df['count'], replace=True)

In [4]:
sdf

,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
623,LIN,624,49360,18.30,35075.39,2.29,0.13,95.13,0.03,2.02,0.40
14531,DINGLER,14530,1881,0.70,72027.50,96.76,0.37,0,0,0.85,1.59
346,THORNTON,347,81191,30.10,28633.51,69.33,26.53,0.37,0.66,1.80,1.31
120,ELLIS,121,181934,67.44,18617.52,72.82,22.60,0.44,0.66,1.64,1.83
1375,BUCKNER,1376,23645,8.77,44372.47,69.30,26.28,0.28,0.83,1.89,1.41
...,...,...,...,...,...,...,...,...,...,...,...
12,TAYLOR,13,720370,267.04,5713.87,67.80,27.67,0.39,0.75,1.78,1.61
1041,MARINO,1042,30735,11.39,41057.45,87.16,0.63,0.90,0.20,1.25,9.86
33285,SPARANO,33248,648,0.24,79491.99,98.30,0,0,0.00,0.00,1.54
2,WILLIAMS,3,1534042,568.66,2137.96,48.52,46.72,0.37,0.78,2.01,1.60


## Assign race by pertcentage

In [5]:
from numpy.random import choice

races = ['white', 'black', 'api', 'hispanic']

def to_race(c):
    w = np.array(c).astype(float)
    if w.sum() == 0:
        return 'white'
    probs = w/w.sum()    
    return choice(races, p=probs)

sdf['race'] = sdf[['pctwhite', 'pctblack', 'pctapi', 'pcthispanic']].apply(lambda c: to_race(c), axis=1)


## Check the correctness of race assignment

In [6]:
df[df.name == 'SMITH']

,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
0,SMITH,1,2376206,880.85,880.85,73.35,22.22,0.40,0.85,1.63,1.56


In [7]:
xdf = sdf[sdf.name=='SMITH'].groupby(['race']).agg({'name': 'count'})
xdf * 100 / xdf.sum()

,name
race,
api,0.437525
black,22.608873
hispanic,1.739927
white,75.213675


In [8]:
# Additional features
sdf['name_last'] = sdf.name.str.title()
sdf.groupby('race').agg({'name_last': 'count'})

,name_last
race,
api,35590
black,125705
hispanic,127898
white,710807


In [9]:
len(sdf)

1000000

## Preprocessing the input data

In [10]:
# only last name in Census data
sdf['name_last_name_first'] = sdf['name_last']

# build n-gram list
vect = CountVectorizer(analyzer='char', max_df=0.3, min_df=3, ngram_range=(NGRAMS, NGRAMS), lowercase=False) 
#vect = CountVectorizer(analyzer='char', ngram_range=(2, 2), lowercase=False) 
a = vect.fit_transform(sdf.name_last_name_first)
vocab = vect.vocabulary_
len(vocab)

955

In [11]:
import operator
sorted_vocab = sorted(vocab.items(), key=operator.itemgetter(1))
cols = list(map(operator.itemgetter(0), sorted_vocab))

In [12]:
count_df = pd.DataFrame(a.todense(), columns=cols)
count_df

,Aa,Ab,Ac,Ad,Ae,Af,Ag,Ah,Ai,Aj,...,zp,zq,zr,zs,zt,zu,zv,zw,zy,zz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
999996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
999997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
999998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
count_df.sum().sort_values(ascending=False).describe()

count       955.000000
mean       5556.781152
std       12700.866095
min           3.000000
25%          89.500000
50%         946.000000
75%        5495.500000
max      175538.000000
dtype: float64

In [14]:
pd.set_option('display.max_rows', 20)
count_df.sum().sort_values(ascending=False)

er    175538
on    123764
an    106961
ar    100367
ll     86325
       ...  
jh         3
Xe         3
Zv         3
Cn         3
qv         3
Length: 955, dtype: int64

In [15]:
# sort n-gram by freq (highest -> lowest)
words = []
for b in vocab:
    c = vocab[b]
    #print(b, c, a[:, c].sum())
    words.append((a[:, c].sum(), b))
    #break
words = sorted(words, reverse=True)
words_list = ['UNK']
words_list.extend([w[1] for w in words])
num_words = len(words_list)
print("num_words = %d" % num_words)


def find_ngrams(text, n):
    a = zip(*[text[i:] for i in range(n)])
    wi = []
    for i in a:
        w = ''.join(i)
        try:
            idx = words_list.index(w)
        except:
            idx = 0
        wi.append(idx)
    return wi

# build X from index of n-gram sequence
X = np.array(sdf.name_last_name_first.apply(lambda c: find_ngrams(c, NGRAMS)))

# check max/avg feature
X_len = []
for x in X:
    X_len.append(len(x))

max_feature_len = max(X_len)
avg_feature_len = int(np.mean(X_len))

print("Max feature len = %d, Avg. feature len = %d" % (max_feature_len, avg_feature_len))
y = np.array(sdf.race.astype('category').cat.codes)

# Split train and test dataset
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

num_words = 956
Max feature len = 14, Avg. feature len = 5


In [16]:
len(vocab)

955

## Train a LSTM model

ref: http://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [17]:
'''The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
Notes:

- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.

- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Activation
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import load_model

max_features = num_words # 20000
feature_len = 20 # avg_feature_len # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=feature_len)
X_test = sequence.pad_sequences(X_test, maxlen=feature_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

800000 train sequences
200000 test sequences
Pad sequences (samples x time)
X_train shape: (800000, 20)
X_test shape: (200000, 20)
4 classes
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (800000, 4)
y_test shape: (200000, 4)


In [18]:
print('Build model...')

model = Sequential()
model.add(Embedding(num_words, 32, input_length=feature_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 32)            30592     
_________________________________________________________________
lstm (LSTM)                  (None, 128)               82432     
_________________________________________________________________
dense (Dense)                (None, 4)                 516       
Total params: 113,540
Trainable params: 113,540
Non-trainable params: 0
_________________________________________________________________
None


2021-12-20 14:50:35.627956: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-20 14:50:35.628132: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-20 14:50:35.628152: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-server-2): /proc/driver/nvidia/version does not exist
2021-12-20 14:50:35.628417: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS,
          validation_split=0.1, verbose=1)
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size, verbose=1)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Epoch 1/15


2021-12-20 14:50:36.001889: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-20 14:50:36.002391: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2250000000 Hz


22500/22500 [==============================] - 452s 19ms/step - loss: 0.5997 - accuracy: 0.7959 - val_loss: 0.5380 - val_accuracy: 0.8135
Epoch 2/15
22500/22500 [==============================] - 426s 19ms/step - loss: 0.5327 - accuracy: 0.8165 - val_loss: 0.5176 - val_accuracy: 0.8199
Epoch 3/15
22500/22500 [==============================] - 424s 19ms/step - loss: 0.5143 - accuracy: 0.8218 - val_loss: 0.5077 - val_accuracy: 0.8224
Epoch 4/15
22500/22500 [==============================] - 424s 19ms/step - loss: 0.5070 - accuracy: 0.8234 - val_loss: 0.5037 - val_accuracy: 0.8238
Epoch 5/15
22500/22500 [==============================] - 424s 19ms/step - loss: 0.4992 - accuracy: 0.8256 - val_loss: 0.4993 - val_accuracy: 0.8247
Epoch 6/15
22500/22500 [==============================] - 425s 19ms/step - loss: 0.4963 - accuracy: 0.8266 - val_loss: 0.4966 - val_accuracy: 0.8256
Epoch 7/15
22500/22500 [==============================] - 423s 19ms/step - loss: 0.4982 - accuracy: 0.8255 - val_loss

## Confusion Matrix

In [20]:
p = model.predict(X_test, verbose=2) # to predict probability
y_pred = np.argmax(p, axis=-1)
target_names = list(sdf.race.astype('category').cat.categories)
print(classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

6250/6250 - 27s
              precision    recall  f1-score   support

         api       0.88      0.64      0.74      7118
       black       0.60      0.05      0.09     25141
    hispanic       0.86      0.84      0.85     25580
       white       0.82      0.97      0.89    142161

    accuracy                           0.83    200000
   macro avg       0.79      0.63      0.65    200000
weighted avg       0.80      0.83      0.78    200000

[[  4572     17    612   1917]
 [    92   1287    178  23584]
 [   103     40  21561   3876]
 [   404    809   2657 138291]]


## Save model

In [21]:
model.save('./census/lstm/census%s_ln_lstm.h5' % YEAR)

In [22]:
words_df = pd.DataFrame(words_list, columns=['vocab'])
words_df.to_csv('./census/lstm/census%s_ln_vocab.csv' % YEAR, index=False, encoding='utf-8')

In [23]:
xdf.to_csv('./census/lstm/census%s_race.csv' % YEAR, columns=[])